<a href="https://colab.research.google.com/github/fintechsteve/modeling-volatility/blob/master/Part_04a_Understanding_Turbulence_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 04 (a): Understanding Turbulence Signals

### In this section you will:


*   Learn about Turbulence and its use in investment management
*   Use a matrix of currency returns to calculate turbulence.
*   Explore the time series properties of turbulence

### Intro

Turbulence is a heuristic measure of volatility made popular by Chow et al. (1999) in "[Optimal Portfolios in Good Times and Bad](https://doi.org/10.2469/faj.v55.n3.2273)" based on the distance metric proposed by Mahalanobis in "[On the generalised distance in statistics](http://insa.nic.in/writereaddata/UpLoadedFiles/PINSA/Vol02_1936_1_Art05.pdf)".

In statistical terms, the metric $d_t$ describes how abnormal a set of returns are by comparing them against a normal approximation of the historic distribution of returns:

\begin{equation*}
d_t = \left( r_t - \mathbf{\mu} \right) \mathbf{\Sigma}^{-1} \left( r_t - \mathbf{\mu} \right)'
\end{equation*}

Where $\mathbf{\Sigma}$ and $\mathbf{\mu}$ are the covariance and mean return measured over the past $n$ days.

## Import all necessary libraries

For this piece, we will need the following packages to be available to our environment:

*   Numpy and Pandas (For data manipulation)
*   DateTime (For basic date manipulation)
*   Matplotlib (For timeseries vizualization)

If the packages are not available, install the with "pip install X"

In [ ]:
import numpy as np, pandas as pd
import statistics
from datetime import datetime, timedelta
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

### Read in data from previously stored returns.pkl file



In [ ]:
with open('./returns.pkl', 'rb') as f:
    returns = pickle.load(f)
    f.close()
returns.head(10)

### Calculate a covariance and mean of returns
For now, we will calcualte turbulence using the full sample covariance and mean.

df.cov() and df.mean() are needed.

In [ ]:
retcov = returns.cov()
retcov

In [ ]:
retmean = returns.mean()
retmean

### Calculate turbulence by demeaning and applying inverse covariance matrix

np.linalg.pinv is a quick way of calculating the inverse of a covariance matrix.
np.einsum is a useful tool for matrix algebra. einsum('ij,ij->i',A, B) calculates the sumproduct of rows of A and B.

In [ ]:
retinvcov = np.linalg.pinv(retcov)
turbulence = pd.DataFrame(np.einsum('ij,ij->i',(returns - retmean),(returns-retmean)@retinvcov),index=returns.index,columns=['daily'])
turbulence.head()

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10, 5))
turbulence.plot(subplots=True, ax=axes, sharex=False, sharey=False, legend=False)

### Demonstrate that contemporaneously turbulence is concurrrent with high index volatility



In [ ]:
dxy_weight = [0, 0.119, 0.036, 0, 0.136, 0.576, 0, 0, 0.091]
dxy = returns.dot(dxy_weight)

In [ ]:
highturbulence = turbulence['daily']>np.percentile(turbulence['daily'],80)
pd.DataFrame([[np.sqrt(261)*dxy[highturbulence].std(), np.sqrt(261)*dxy[~highturbulence].std()], [sum(highturbulence), sum(~highturbulence)]], index=['std','N'], columns=['High Turb','Low Turb'])

### Demonstrate that turbulence leads to persistently high volatility days

In [ ]:
leading_dxy = pd.DataFrame([dxy.shift(-i) for i in range(0, 11)]).T
subsequent_vol = pd.DataFrame([leading_dxy[highturbulence].std(), leading_dxy[~highturbulence].std()],index=['High Turb','Low Turb']).T
subsequent_vol

In [ ]:
subsequent_vol.plot()